In [1]:
import pandas as pd
import numpy as np
import math

features = pd.read_csv(r"G:\Signature Verification Project\MCYT 100 Dataset\Processed\test_gen.csv")



## finding the position of pen-tip
Xs = features["x"].tolist()
Ys = features["y"].tolist()

Rs = []

for i in range(len(Xs)):
    Rs.append(math.sqrt(Xs[i]**2 + Ys[i]**2))

features["R"] = Rs ## "R" is the position column



## angle of "R"

theta = []

for i in range(len(Xs)):
    try:
        theta.append(np.arctan(Ys[i]/Xs[i]))
    except ZeroDivisionError:
        theta.append(0)

features["Angle"] = theta



## finding change in X direction
features['del_x'] = np.where(features.id == features.id.shift(1), features.x.diff(), 0)
features['del_x'] = features.groupby("id")["del_x"].shift(-1)


## finding change in Y direction
features['del_y'] = np.where(features.id == features.id.shift(1), features.y.diff(), 0)
features['del_y'] = features.groupby("id")["del_y"].shift(-1)



## Magnitude and angle of displacement vector

del_x_l = features.del_x.tolist()
del_y_l = features.del_y.tolist()

disp = [] ## magnitude
disp_theta = [] ## angle

for i in range(len(del_x_l)):
    
    ## magnitude
    disp.append(math.sqrt(del_x_l[i]**2 + del_y_l[i]**2))
    
    ## angle
    try:
        disp_theta.append(np.arctan(del_y_l[i]/del_x_l[i]))
    except ZeroDivisionError:
        disp_theta.append(0)
    
features["displacement"] = disp
features["disp_angle"] = disp_theta



## finding time
features["Time"] = features.groupby(["id"]).cumcount()+1
features["Time"] = 10*features["Time"]


# In[33]:


## finding change in Time
features['del_t'] = np.where(features.id == features.id.shift(1), features.Time.diff(), 0)
features['del_t'] = features.groupby("id")["del_t"].shift(-1)

features['del_t'] = features['del_t'].fillna(value=10)

# In[35]:


del_x_l = features.del_x.tolist()
del_y_l = features.del_y.tolist()
del_t_l = features.del_t.tolist()

vel_x = [] ## velocity in x-direction
vel_y = [] ## velocity in y-direction

for i in range(len(del_x_l)):
    vel_x.append(del_x_l[i]/del_t_l[i])
    vel_y.append(del_y_l[i]/del_t_l[i])
    
features["vel_x"] = vel_x
features["vel_y"] = vel_y



# In[38]:


res_vel = []  ## resultant velocity 
dir_res_vel = []  ## direction of the resultant velocity

for i in range(len(vel_x)):
    res_vel.append(math.sqrt(vel_x[i]**2 + vel_y[i]**2))
    
    try:
        dir_res_vel.append(np.arctan(vel_y[i]/vel_x[i]))
    except ZeroDivisionError:
        dir_res_vel.append(0)
        
features["res_vel"] = res_vel
features["dir_res_vel"] = dir_res_vel



## finding change in displacement angle
features['del_disp_angle'] = np.where(features.id == features.id.shift(1), features.disp_angle.diff(), 0)
features['del_disp_angle'] = features.groupby("id")["del_disp_angle"].shift(-1)
## converting it to list
del_disp_angle = features.del_disp_angle.tolist()

angular_vel = [] ## angular velocity

for i in range(len(del_disp_angle)):
    angular_vel.append(del_disp_angle[i]/del_t_l[i])

features["angular_vel"] = angular_vel


del_vel_x = features.vel_x.tolist()
del_vel_y = features.vel_y.tolist()
del_t_l = features.del_t.tolist()

acceleration_x = [] ## acceleration in x-dir
acceleration_y = [] ## acceleration in y-dir
res_acceleration = [] ## resultant acceleration

for i in range(len(del_t_l)):
    acceleration_x.append(del_vel_x[i]/del_t_l[i])
    acceleration_y.append(del_vel_y[i]/del_t_l[i])
    
    res_acceleration.append(math.sqrt(acceleration_x[i]**2 + acceleration_y[i]**2))
    
features["acceleration_x"] = acceleration_x
features["acceleration_y"] = acceleration_y
features["res_acceleration"] = res_acceleration

del del_vel_x, del_vel_y, del_t_l, acceleration_x, acceleration_y, res_acceleration



angular_vel = features.angular_vel.tolist()
displacement = features.displacement.tolist()
res_acceleration = features.res_acceleration.tolist()

centripetal_accln = [] ## centripetal acceleration
tangential_accln = [] ## tangential acceleration

for i in range(len(angular_vel)):
    centripetal_accln.append((angular_vel[i]**2)*displacement[i])
    
    tangential_accln.append(math.sqrt(abs(res_acceleration[i]**2 - centripetal_accln[i]**2)))
    
features["centripetal_accln"] = centripetal_accln
features["tangential_accln"] = tangential_accln

del angular_vel, res_acceleration, displacement, centripetal_accln, tangential_accln


new = features[['id', 'x', 'y', 'pressure', 'azimuth', 'elevation', 'R', 'displacement', 'res_vel', 'angular_vel', 'res_acceleration', 'centripetal_accln', 'tangential_accln']]

new[["displacement", "res_vel", "angular_vel", "res_acceleration", "centripetal_accln", "tangential_accln"]] = new[["displacement", "res_vel", "angular_vel", "res_acceleration", "centripetal_accln", "tangential_accln"]].fillna(0)

new = new.rename(columns = {"id":"ID", "x":"X", "y":"Y",  "pressure":"PRESSURE", "azimuth":"AZIMUTH", "elevation":"ELEVATION", 
                      "R":"POSITION", "displacement":"DISPLACEMENT", "res_vel":"VELOCITY", "angular_vel":"ANGULAR_VELOCITY", 
                      "res_acceleration":"ACCELERATION", "centripetal_accln":"CENTRIPETAL_ACCELERATION", 
                      "tangential_accln":"TANGENTIAL_ACCELERATION"})

new.to_csv(r"G:\Signature Verification Project\MCYT 100 Dataset\Processed\new_processed\test_gen.csv", index = False)

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [5]:
new.head()

,ID,X,Y,PRESSURE,AZIMUTH,ELEVATION,ID,DISPLACEMENT,VELOCITY,ANGULAR_VELOCITY,ACCELERATION,CENTRIPETAL_ACCELERATION,TANGENTIAL_ACCELERATION
0,0060v00,1235.0,8144.0,86.0,67.0,135.0,0060v00,36.496575,3.649658,-0.140565,0.364966,0.721116,0.621939
1,0060v00,1241.0,8180.0,195.0,67.0,135.0,0060v00,9.000000,0.900000,0.000000,0.090000,0.000000,0.090000
2,0060v00,1241.0,8189.0,296.0,67.0,135.0,0060v00,8.000000,0.800000,0.000000,0.080000,0.000000,0.080000
3,0060v00,1241.0,8197.0,384.0,65.0,135.0,0060v00,7.000000,0.700000,-0.000000,0.070000,0.000000,0.070000
4,0060v00,1241.0,8204.0,450.0,65.0,135.0,0060v00,5.000000,0.500000,0.000000,0.050000,0.000000,0.050000
